In [519]:
import pandas as pd
import yaml
from sqlalchemy import create_engine
from datetime import timedelta

## Database Connection

In [520]:
with open('config.yml', 'r') as f: #Abrir el archivo en modo de  lectura
    config = yaml.safe_load(f) # Crear un diccionario con lo que hay en el archivo
    config_db_etl = config['bodega'] #Obtener solo la configuración de la bodega
    config_db = config["fuente"] #Obtener solo la configuración de la bodega

In [521]:
# Construct the database URL
url_db_etl = (f"{config_db_etl['driver']}://{config_db_etl['user']}:{config_db_etl['password']}@{config_db_etl['host']}:"
           f"{config_db_etl['port']}/{config_db_etl['db']}")
url_db = (f"{config_db['driver']}://{config_db['user']}:{config_db['password']}@{config_db['host']}:"
           f"{config_db['port']}/{config_db['db']}")

In [522]:
# Create the SQLAlchemy Engine
etl_conn = create_engine(url_db_etl)
olap_conn = create_engine(url_db)

In [523]:
pd.set_option('display.max_columns', None)  # Mostrar todas las columnas sin saltos
pd.set_option('display.width', 1000)  

## Extraction


In [524]:
fase_servicio = pd.read_sql_table('dim_fase_servicio', etl_conn) 

In [525]:
total_nan = fase_servicio.isna().sum()
print(total_nan)
print(fase_servicio.columns)

key_fase_servicio         0
id                        0
fecha                     0
hora                      0
foto                      0
observaciones             1
estado_id                 0
servicio_id               0
es_prueba                 0
foto_binary          128132
dtype: int64
Index(['key_fase_servicio', 'id', 'fecha', 'hora', 'foto', 'observaciones', 'estado_id', 'servicio_id', 'es_prueba', 'foto_binary'], dtype='object')


In [526]:
print(len(fase_servicio))

128402


## calculo tiempo por fase_servicio

In [ ]:
import pandas as pd

# Ordenar por 'servicio_id' y 'fecha'
fase_servicio = fase_servicio.sort_values(by=['servicio_id', 'fecha'])

# Asegurarse de que la columna 'hora' esté en formato timedelta
fase_servicio['hora'] = pd.to_timedelta(fase_servicio['hora'].astype(str))

# Calcular la diferencia en días entre fases consecutivas dentro de cada servicio
fase_servicio['dias_de_demora'] = fase_servicio.groupby('servicio_id')['fecha'].transform(lambda x: x.diff().dt.days)

# asignar el resultado a la columna directamente para evitar errores jeje
fase_servicio['dias_de_demora'] = fase_servicio['dias_de_demora'].fillna(0)

# Calcular la diferencia en horas entre fases consecutivas dentro de cada servicio
fase_servicio['hora_de_demora'] = fase_servicio.groupby('servicio_id')['hora'].transform(lambda x: x.diff())

# Corregir las horas negativas: Si la hora actual es menor que la anterior, agregar 24 horas
fase_servicio['hora_de_demora'] = fase_servicio['hora_de_demora'].apply(lambda x: x if x >= pd.Timedelta(0) else x + pd.Timedelta(days=1))

# Convertir la diferencia en horas
fase_servicio['hora_de_demora'] = fase_servicio['hora_de_demora'].dt.total_seconds() / 3600

# Lo mismo para la columna 'hora_de_demora'
fase_servicio['hora_de_demora'] = fase_servicio['hora_de_demora'].fillna(0)

# Convertir las columnas 'hora' y 'hora_de_demora' a su formato de horas
fase_servicio['hora'] = fase_servicio['hora'].dt.total_seconds() / 3600
fase_servicio['hora_de_demora'] = fase_servicio['hora_de_demora'].apply(lambda x: round(x, 2))  # Redondear a 2 decimales


fase_servicio = fase_servicio[["key_fase_servicio", "fecha", "hora", "estado_id", "servicio_id", "dias_de_demora", "hora_de_demora"]]


fase_servicio.head(20)


       key_fase_servicio      fecha       hora  estado_id  servicio_id  dias_de_demora  hora_de_demora
301                  301 2023-09-19  16.371667          1            7             0.0            0.00
667                  667 2023-10-13  17.855556          2            7            24.0            1.48
686                  686 2023-10-31  12.046901          4            7            18.0           18.19
687                  687 2023-10-31  17.131944          5            7             0.0            5.09
688                  688 2023-10-31  12.266667          6            7             0.0           19.13
302                  302 2023-09-19  16.501389          1            8             0.0            0.00
792                  792 2023-12-20  20.245278          2            8            92.0            3.74
5902                5902 2024-02-14  15.571667          4            8            56.0           19.33
34483              34483 2024-04-09  16.143056          5            8   

## Calculo atributo tiempo promedio general

In [ ]:


# Calcular el tiempo total transcurrido desde la primera hasta la última fase para cada servicio
fase_servicio['tiempo_total'] = fase_servicio.groupby('servicio_id')['fecha'].transform(lambda x: (x.max() - x.min()).days)

# Calcular el número de fases por servicio
fase_servicio['numero_fases'] = fase_servicio.groupby('servicio_id')['fecha'].transform('count')

# Calcular el tiempo promedio por fase en días (convertir de horas a días)
fase_servicio['tiempo_promedio_dias'] = fase_servicio['tiempo_total'] // fase_servicio['numero_fases']

# Calcular el tiempo promedio en horas (total de horas)
fase_servicio['tiempo_promedio_horas'] = (fase_servicio['tiempo_total'] * 24) // fase_servicio['numero_fases']


fase_servicio.head(10)



       key_fase_servicio      fecha       hora  estado_id  servicio_id  dias_de_demora  hora_de_demora  tiempo_total  numero_fases  tiempo_promedio_dias  tiempo_promedio_horas
301                  301 2023-09-19  16.371667          1            7             0.0            0.00            42             5                     8                    201
667                  667 2023-10-13  17.855556          2            7            24.0            1.48            42             5                     8                    201
686                  686 2023-10-31  12.046901          4            7            18.0           18.19            42             5                     8                    201
687                  687 2023-10-31  17.131944          5            7             0.0            5.09            42             5                     8                    201
688                  688 2023-10-31  12.266667          6            7             0.0           19.13            42    

## Load

In [529]:
fase_servicio.to_sql("hecho_servicio_fase", etl_conn, if_exists="replace", index_label="key_servicio_fase") 

402